# Basic liquid handling

In this notebook, you will learn how to use PyHamilton to move water from one range of wells to another.

**Note: before running this notebook, you should have**:

- Installed PyHamilton and the USB driver as described in [the installation guide](installation).
- Connected the Hamilton to your computer using the USB cable.

Video of what this code does:

<iframe width="640" height="360" src="https://www.youtube.com/embed/NN6ltrRj3bU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Setting up a connection with the robot

Start by importing the {class}`~pyhamilton.liquid_handling.liquid_handler.LiquidHandler` class, which will serve as a front end for all liquid handling operations.

Backends serve as communicators between between `LiquidHandler`s and the actual hardware. Since we are using a Hamilton STAR, we also import the {class}`~pyhamilton.liquid_handling.backends.STAR` backend.

In [1]:
from pyhamilton.liquid_handling import LiquidHandler
from pyhamilton.liquid_handling.backends import STAR

Create a new liquid handler using `STAR` as its backend

In [2]:
backend = STAR()
lh = LiquidHandler(backend=backend)

The final step is to open communication with the robot. This is done using the {func}`~pyhamilton.liquid_handling.LiquidHandler.setup` method.

In [3]:
lh.setup()

INFO:pyhamilton.liquid_handling.backends.hamilton:Finding Hamilton USB device...
INFO:pyhamilton.liquid_handling.backends.hamilton:Found Hamilton USB device.
INFO:pyhamilton.liquid_handling.backends.hamilton:Found endpoints. 
Write:
       ENDPOINT 0x2: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x2 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :   0x40 (64 bytes)
       bInterval        :    0x0 
Read:
       ENDPOINT 0x81: Bulk IN ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :   0x81 IN
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :   0x40 (64 bytes)
       bInterval        :    0x0
INFO:pyhamilton.liquid_handling.backends.hamilton:Sent command: C0QWid0004
INFO:pyhamilton.liquid_handling.backends.hamilton:Received response: C0QWid0004er00/00qw

## Creating the deck layout

Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyHamilton package. This notebook uses the following resources:

- {class}`~pyhamilton.liquid_handling.resources.ml_star.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pyhamilton.liquid_handling.resources.ml_star.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pyhamilton.liquid_handling.resources.corning_costar.plates.Cos_96_DW_1mL` wells
- {class}`~pyhamilton.liquid_handling.resources.ml_star.tips.STF_L` tips

In [4]:
from pyhamilton.liquid_handling.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    STF_L
)

Then create a tip carrier named `tip carrier`, which will contain tips at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [5]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = STF_L(name='tips_01')
tip_car[1] = STF_L(name='tips_02')
tip_car[2] = STF_L(name='tips_03')
tip_car[3] = STF_L(name='tips_04')
tip_car[4] = STF_L(name='tips_05')

Use {func}`~pyhamilton.liquid_handling.LiquidHandler.assign_resource` to assign the tip carrier to the liquid handler. All resources contained by this carrier will also be assigned.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [6]:
lh.assign_resource(tip_car, rails=3)

Repeat this for the plates.

In [7]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')

In [8]:
lh.assign_resource(plt_car, rails=9)

Let's look at a summary of the deck layout using {func}`~pyhamilton.liquid_handling.LiquidHandler.summary`.

In [9]:
lh.summary()

Rail     Resource                   Type                Coordinates (mm)
(3)  ├── tip carrier                TIP_CAR_480_A00     (145.000, 063.000, 100.000)
     │   ├── tips_01                STF_L               (162.900, 145.800, 164.450)
     │   ├── tips_02                STF_L               (162.900, 241.800, 164.450)
     │   ├── tips_03                STF_L               (162.900, 337.800, 164.450)
     │   ├── tips_04                STF_L               (162.900, 433.800, 164.450)
     │   ├── tips_05                STF_L               (162.900, 529.800, 164.450)
     │
(9)  ├── plate carrier              PLT_CAR_L5AC_A00    (280.000, 063.000, 100.000)
     │   ├── plate_01               Cos_96_DW_1mL       (298.000, 146.000, 187.150)
     │   ├── <empty>
     │   ├── <empty>
     │   ├── <empty>
     │   ├── <empty>


## Picking up tips

Picking up tips is as easy as specifying the name of the tips resource and for each channel the locations of the tips within that resource. Here, the parameter names are included, but that is not required: `h.pickup_tips("tips_01", "A1", "B1", "C1")` would be the same.

In [10]:
lh.pickup_tips("tips_01", channel_1="A1", channel_2="B1", channel_3="C1")

INFO:pyhamilton.liquid_handling.backends.hamilton:Sent command: C0TPid0050xp01629 01629 01629&yp2418 2328 2238 0000&tm1 1 1 0&tt01tp2244tz2164th2450td0
INFO:pyhamilton.liquid_handling.backends.hamilton:Received response: C0TPid0050er00/00


{'id': 50}

## Aspirating and dispensing

PyHamilton has many built-in liquid classes, which are used to update the default parameters of the liquid handling operations. This includes correcting the volume to aspirate or dispense, according to a liquid's properties. Here, we will use the default {class}`~pyhamilton.liquid_handling.liquid_classes.StandardVolumeFilter_Water_DispenseSurface_Part_no_transport_vol` liquid class.

In [12]:
from pyhamilton.liquid_handling.liquid_classes import StandardVolumeFilter_Water_DispenseSurface_Part_no_transport_vol

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pyhamilton.liquid_handling.LiquidHandler.aspirate` and {func}`~pyhamilton.liquid_handling.LiquidHandler.dispense` methods.

In [13]:
lh.aspirate("plate_01", ("A1", 100.0), ("B1", 50.0), ("C1", 200.0),
  liquid_class=StandardVolumeFilter_Water_DispenseSurface_Part_no_transport_vol)

INFO:pyhamilton.liquid_handling.backends.hamilton:Sent command: C0ASid0052at0&tm1 1 1 0&xp02980 02980 02980 00000&yp1460 1370 1280 0000&th2450te2450lp2321 2321 2321&ch000 000 000&zl1881 1881 1881&po0100 0100 0100&zu0032 0032 0032&zr06180 06180 06180&zx1871 1871 1871&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&av01072 00536 02144&as1000 1000 1000&ta000 000 000&ba0000 0000 0000&oa000 000 000&lm0 0 0&ll1 1 1&lv1 1 1&zo000 000 000&ld00 00 00&de0020 0020 0020&wt10 10 10&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms1000 1000 1000&mh0000 0000 0000&gi000 000 000&gj0gk0lk0 0 0&ik0000 0000 0000&sd0500 0500 0500&se0500 0500 0500&sz0300 0300 0300&io0000 0000 0000&il00000 00000 00000&in0000 0000 0000&
INFO:pyhamilton.liquid_handling.backends.hamilton:Received response: C0ASid0052er00/00


'C0ASid0052er00/00'

After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [15]:
lh.dispense("plate_01", ("D1", 100.0), ("E1", 50.0), ("F1", 200.0))

INFO:pyhamilton.liquid_handling.backends.hamilton:Sent command: C0DSid0053dm2 2 2&tm1 1 1 0&xp02980 02980 02980 00000&yp1190 1100 1010 0000&zx1871 1871 1871&lp2321 2321 2321&zl1881 1881 1881&po0100 0100 0100&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&zu0032 0032 0032&zr06180 06180 06180&th2450te2450dv01072 00536 02144&ds1200 1200 1200&ss0050 0050 0050&rv000 000 000&ta000 000 000&ba0000 0000 0000&lm0 0 0&dj00zo000 000 000&ll1 1 1&lv1 1 1&de0020 0020 0020&wt00 00 00&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms0010 0010 0010&mh0000 0000 0000&gi000 000 000&gj0gk0
INFO:pyhamilton.liquid_handling.backends.hamilton:Received response: C0DSid0053er00/00


{'id': 53}

Let's move the liquid back to the original wells.

In [17]:
lh.aspirate("plate_01", ("D1", 100.0), ("E1", 50.0), ("F1", 200.0))
lh.dispense("plate_01", ("A1", 100.0), ("B1", 50.0), ("C1", 200.0))

INFO:pyhamilton.liquid_handling.backends.hamilton:Sent command: C0ASid0054at0&tm1 1 1 0&xp02980 02980 02980 00000&yp1190 1100 1010 0000&th2450te2450lp2321 2321 2321&ch000 000 000&zl1881 1881 1881&po0100 0100 0100&zu0032 0032 0032&zr06180 06180 06180&zx1871 1871 1871&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&av01072 00536 02144&as1000 1000 1000&ta000 000 000&ba0000 0000 0000&oa000 000 000&lm0 0 0&ll1 1 1&lv1 1 1&zo000 000 000&ld00 00 00&de0020 0020 0020&wt10 10 10&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms1000 1000 1000&mh0000 0000 0000&gi000 000 000&gj0gk0lk0 0 0&ik0000 0000 0000&sd0500 0500 0500&se0500 0500 0500&sz0300 0300 0300&io0000 0000 0000&il00000 00000 00000&in0000 0000 0000&
INFO:pyhamilton.liquid_handling.backends.hamilton:Received response: C0ASid0054er00/00
INFO:pyhamilton.liquid_handling.backends.hamilton:Sent command: C0DSid0055dm2 2 2&tm1 1 1 0&xp02980 02980 02980 00000&yp1460 1370 1280 0000&zx1871 1871 1871&lp2321 2321 2321&zl1881 1881 1881&po0100 0100 0100&ip0000 

## Discarding tips

Finally, you can discard tips by using the {func}`~pyhamilton.liquid_handling.LiquidHandler.discard_tips` method.

In [18]:
lh.discard_tips("tips_01", "A1", "B1", "C1")

INFO:pyhamilton.liquid_handling.backends.hamilton:Sent command: C0TRid0056xp01629 01629 01629 00000&yp2418 2328 2238 0000&tm1 1 1 0 0&tt01tp2244tz2164th2450ti0
INFO:pyhamilton.liquid_handling.backends.hamilton:Received response: C0TRid0056er00/00kz381 356 365 000 000 000 000 000vz386 360 369 000 000 000 000 000


'C0TRid0056er00/00kz381 356 365 000 000 000 000 000vz386 360 369 000 000 000 000 000'